<a href="https://colab.research.google.com/github/skrb33/language-exercise/blob/main/notebooks/Attribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone repo to access resources

In [2]:
!git clone https://github.com/idealo/cnn-exposed.git

Cloning into 'cnn-exposed'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 58 (delta 0), reused 0 (delta 0), pack-reused 55 (from 1)
Receiving objects: 100% (58/58), 71.30 MiB | 39.29 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [1]:
cd cnn-exposed/

[Errno 2] No such file or directory: 'cnn-exposed/'
/content


# Setup

In [3]:
# import packages
import os
import pathlib
import requests

import numpy as np
import matplotlib.image as mpimg

from PIL import Image
from io import BytesIO
from skimage import feature, transform
from matplotlib.pyplot import figure
from matplotlib import pyplot as plt
%matplotlib inline

# import keras dependencies
from keras.models import Model
from keras.applications import MobileNet as CNN
from keras.applications.mobilenet import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.utils import to_categorical
import keras.backend as K

### Declare some helper functions

In [4]:
def plot_single_image(image_path, fig_size=(10, 10), dpi=100):
    figure(figsize=fig_size, dpi=dpi)
    img = mpimg.imread(image_path)
    plt.imshow(img)
    plt.grid(False)
    plt.axis('off')
    plt.show()


# Plotting function for saliency maps
def plot_custom(data, xi=None, cmap='RdBu_r', axis=plt, percentile=100, dilation=3.0, alpha=0.8):
    dx, dy = 0.05, 0.05
    xx = np.arange(0.0, data.shape[1], dx)
    yy = np.arange(0.0, data.shape[0], dy)
    xmin, xmax, ymin, ymax = np.amin(xx), np.amax(xx), np.amin(yy), np.amax(yy)
    extent = xmin, xmax, ymin, ymax
    cmap_xi = plt.get_cmap('Greys_r')
    cmap_xi.set_bad(alpha=0)
    overlay = None
    if xi is not None:
        # Compute edges (to overlay to heatmaps later)
        xi_greyscale = xi if len(xi.shape) == 2 else np.mean(xi, axis=-1)
        in_image_upscaled = transform.rescale(xi_greyscale, dilation, mode='constant')
        edges = feature.canny(in_image_upscaled).astype(float)
        edges[edges < 0.5] = np.nan
        edges[:5, :] = np.nan
        edges[-5:, :] = np.nan
        edges[:, :5] = np.nan
        edges[:, -5:] = np.nan
        overlay = edges

    abs_max = np.percentile(np.abs(data), percentile)
    abs_min = -abs_max

    if len(data.shape) == 3:
        data = np.mean(data, 2)
    axis.imshow(data, extent=extent, interpolation='bicubic', cmap=cmap, vmin=abs_min, vmax=abs_max)
    if overlay is not None:
        axis.imshow(overlay, extent=extent, interpolation='bicubic', cmap=cmap_xi, alpha=alpha)
    axis.axis('off')
    return axis


def plot_comparison(target_image_path, map_array, title=''):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(26, 20))

    img_orig = Image.open(target_image_path).resize((224, 224))
    xi = (map_array[0,:] - np.min(map_array[0,:]))
    xi /= np.max(xi)

    ax = axes.flatten()[0]
    ax.imshow(img_orig)
    ax.set_title('Original', fontdict={'fontsize': 20})
    ax.axis('off')

    plot_custom(attributions[0], xi = xi, axis=axes[1], dilation=.5, percentile=99, alpha=.2).set_title(title, fontdict={'fontsize': 20})
    plt.show()


def plot_gradcam(original_image_path, loaded_image, grads):
    fig, axs = plt.subplots(1, 2, figsize=(16, 10), constrained_layout=True)

    img_orig = Image.open(original_image_path).resize((224, 224))
    axs[0].imshow(img_orig, aspect='auto')
    axs[0].grid(False)

    axs[1].imshow(overlay(grads, loaded_image), aspect='auto')
    axs[1].grid(False)

In [5]:
path_resources = pathlib.Path('resources/attribution/')

# Attribution approaches

Several approaches:


1.   **Perturbation based approaches** - Occlude area of interest to test its effectiveness in changing predictions
2.   **Gradient bases approaches** - Calculation of gradients of output w.r.t. some network variable
  * *Saliency Maps*
  * Guided Backpropagation
  * Deconvolution
3.   **Relevance Score approaches**
  * *Class Activation Map (CAM)*
  * *Grad- CAM*



# Image classification
We wish to classify the below image with true label for the dog breed: Australian Terrier

In [6]:
target_image = os.path.join(path_resources, 'inp_im.png')
plot_single_image(target_image)

FileNotFoundError: [Errno 2] No such file or directory: 'resources/attribution/inp_im.png'

<Figure size 1000x1000 with 0 Axes>

# Saliency Maps
## Deep Inside Convolutional Networks: Visualising Image Classification Models and Saliency Maps
by Simonyan, Vedaldi, Zisserman

(https://arxiv.org/pdf/1312.6034v2.pdf)

### Take gradient of output w.r.t each input

In [11]:
saliency_diagram = os.path.join(path_resources, 'saliency.png')
plot_single_image(saliency_diagram, fig_size=(18, 18))

FileNotFoundError: [Errno 2] No such file or directory: 'resources/attribution/saliency.png'

<Figure size 1800x1800 with 0 Axes>

## Python package: DeepExplain
(https://github.com/marcoancona/DeepExplain#egg=deepexplain)

In [12]:
# Get the model (Mobilenet Pretrained on ImageNet dataset)
model = CNN(include_top=True)

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
# Get the prediction
loaded_image = np.array(image.load_img(target_image, target_size=(224, 224)))
processed_image = preprocess_input(loaded_image)
preds = model.predict(processed_image[np.newaxis, :])
preds_name = decode_predictions(preds)
preds_name

FileNotFoundError: [Errno 2] No such file or directory: 'resources/attribution/inp_im.png'

## Install Deepexplain package for plotting saliency maps

In [14]:
!pip install git+https://github.com/marcoancona/DeepExplain.git#egg=deepexplain

  Cloning https://github.com/marcoancona/DeepExplain.git to /tmp/pip-install-960zfwlq/deepexplain_392ca7d889f64b4c90baa3de5e76eba1
  Running command git clone --filter=blob:none --quiet https://github.com/marcoancona/DeepExplain.git /tmp/pip-install-960zfwlq/deepexplain_392ca7d889f64b4c90baa3de5e76eba1
  Resolved https://github.com/marcoancona/DeepExplain.git to commit 87fb43a13ac2a3b285a030b87df899cc40100c94
  Preparing metadata (setup.py) ... done
  Created wheel for deepexplain: filename=deepexplain-0.3-py3-none-any.whl size=15246 sha256=03b70ddc113a622e7ac9f55a06a388b9085875486557d4ed6a842f586dc8a9ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-4lhqjlft/wheels/c6/3b/f1/c77c05e206a49abec9107da7bad879af005231a97eccc1aa39
Successfully built deepexplain


In [ ]:
# import deepexplain to draw saliency maps
from deepexplain.tensorflow import DeepExplain

In [ ]:
# Get saliency map
# Refer the API documentation for using deepexplain as below
with DeepExplain(session=K.get_session()) as de:
    model = CNN(include_top=True)
    input_tensor = model.layers[0].input
    fModel = Model(inputs=input_tensor, outputs = model.layers[-1].output)
    target_tensor = fModel(input_tensor)
    top_idx = preds.argsort()[::-1]
    ys = to_categorical(top_idx, num_classes=1000)
    xs = np.tile(processed_image, (1, 1, 1, 1))
    attributions = de.explain('saliency', fModel.outputs[0] * ys, fModel.inputs[0], xs)

In [ ]:
plot_comparison(target_image, attributions, title='Saliency map')

## Saliency Map Pros:

*   Fine-grained understanding of relative contribution of pixels

## Saliency Map Cons:
* Propagation of gradient is hard (due to non-linearlities such as relu which makes the gradient discontinuous)

# Class Activation Map (CAM)
## Learning Deep Features for Discriminative Localization
by Zhou et. al

(http://cnnlocalization.csail.mit.edu/Zhou_Learning_Deep_Features_CVPR_2016_paper.pdf)

In [ ]:
cam_diagram = os.path.join(path_resources, 'cam.png')
plot_single_image(cam_diagram, fig_size=(15, 15))

## CAM Pros:

*   No calculation of gradients needed
*   Intuitive to understand

## CAM Cons:
* Bound to a fixed architecture (Conv -> GAP -> Dense)

# Gradient Class Activation Map (Grad-CAM)

## Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization

by Selvaraju et.al (https://arxiv.org/pdf/1610.02391.pdf)

In [ ]:
gradcam_diagram = os.path.join(path_resources, 'gradcam.png')
plot_single_image(gradcam_diagram, fig_size=(14, 14))

In [10]:
# Summary of the MobilNet model (pretrained on ImageNet dataset) that was loaded earlier
model.summary()

NameError: name 'model' is not defined

## Install keras-vis package for obtaining grad-cam

(https://github.com/raghakot/keras-vis)

In [7]:
!pip install keras-vis

In [8]:
# import specific functions from keras-vis package
from vis.utils import utils
from vis.visualization import visualize_cam, overlay

ImportError: cannot import name 'Iterable' from 'collections' (/usr/lib/python3.11/collections/__init__.py)

In [ ]:
# Get layer for which Grad-CAM needs to be obtained ('conv_preds' is the name of
# the convolution layer closest to the output layer. Refer the output of
# model.summary()). Any convolution layer can be visualized (not only the one
# closest to the output)
layer_idx = utils.find_layer_idx(model, 'conv_preds')

In [ ]:
grads = visualize_cam(model, layer_idx, filter_indices=np.argmax(preds), seed_input=processed_image, backprop_modifier=None)

In [ ]:
plot_gradcam(target_image, loaded_image, grads)

# Layerwise Relevance Propagation (LRP)
## On Pixel-Wise Explanations for Non-Linear Classifier Decisions by Layer-Wise Relevance Propagation
by Bach et. al. (https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0130140)

In [ ]:
lrp_diagram_1 = os.path.join(path_resources, 'lrp_new.png')
plot_single_image(lrp_diagram_1, fig_size=(3, 3), dpi=300)
print('source: Montavon et. al., Explaining nonlinear classification decisions with deep Taylor decomposition')

In [ ]:
lrp_diagram_2 = os.path.join(path_resources, 'lrp.png')
plot_single_image(lrp_diagram_2, fig_size=(4, 4), dpi=300)

In [ ]:
# Get Epsilon-LRP heatmap
with DeepExplain(session=K.get_session()) as de:
    model = CNN(include_top=True)
    input_tensor = model.layers[0].input
    fModel = Model(inputs=input_tensor, outputs = model.layers[-1].output)
    target_tensor = fModel(input_tensor)
    top_idx = preds.argsort()[::-1]
    ys = to_categorical(top_idx, num_classes=1000)
    xs = np.tile(processed_image, (1, 1, 1, 1))
    attributions = de.explain('elrp', fModel.outputs[0] * ys, fModel.inputs[0], xs)

In [ ]:
plot_comparison(target_image, attributions, title='Epsilon-LRP')

## Testing out some more images

In [ ]:
image_urls = [
    'https://aristainflorida.com/wp-content/uploads/2017/11/jay-bird-konar-winter-45212-small.jpg',
    'https://cdn.britannica.com/55/31555-131-240223FB.jpg',
    'https://i.pinimg.com/originals/60/a1/3f/60a13ffca856faf182b2ee44cfe59f41.jpg',
]

In [ ]:
images = []
for url in image_urls:
    response = requests.get(url)
    image = Image.open(BytesIO(response.content)).resize((224, 224))
    images.append(np.array(image))

images_pp = preprocess_input(np.array(images))

In [ ]:
# Get predictions
preds = model.predict(images_pp)
preds_name = decode_predictions(preds)

for j, url in enumerate(image_urls):
    print('{}'.format(url))
    for (i, (imagenetID, label, prob)) in enumerate(preds_name[j]):
        print('{}. {}: {:.2f}%'.format(i + 1, label, prob * 100))
    print('\n')

In [ ]:
N_PLOT_PRED = 2 # top-n predictions to plot for each image
K.clear_session()

with DeepExplain(session=K.get_session()) as de:

    model = CNN(include_top=True)
    input_tensor = model.layers[0].input

  # We target the output of the last dense layer (pre-softmax)
  # To do so, create a new model sharing the same layers until the last dense

    fModel = Model(inputs=input_tensor, outputs = model.layers[-1].output)
    target_tensor = fModel(input_tensor)

    for u, url in enumerate(image_urls):
        print('{}:'.format(url))
        top_idx = preds[u].argsort()[-N_PLOT_PRED:][::-1] # Get indices of the top-2 predicted classes

        ys = to_categorical(top_idx, num_classes=1000) # one-hot encode the predicted indices
        xs = np.tile(images_pp[u], (N_PLOT_PRED, 1, 1, 1)) # Duplicate the image N_PLOT_PRED number of times

        # Draw saliency maps and Epsilon-LRP heatmap
        attributions = {
            'Saliency maps': de.explain('saliency', fModel.outputs[0] * ys, fModel.inputs[0], xs),
            'Epsilon-LRP': de.explain('elrp', fModel.outputs[0] * ys, fModel.inputs[0], xs),
        }
        print ('Done!')

        # Plotting Function
        n_cols = int(len(attributions)) + 1
        n_rows = 1

        for i, xi in enumerate(xs):
            print('{}: {:.2f}%'.format(preds_name[u][i][1], preds_name[u][i][2] * 100))

            fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(5*n_cols, 5*n_rows))

            xi = (xi - np.min(xi))
            xi /= np.max(xi)
            ax = axes.flatten()[0]
            ax.imshow(images[u])
            ax.set_title('Original')
            ax.axis('off')

            for j, a in enumerate(attributions):
                axj = axes.flatten()[j + 1]
                plot_custom(attributions[a][i], xi = xi, axis=axj, dilation=.5, percentile=99, alpha=.2).set_title(a)
        plt.show()
        plt.close()